In [1]:
# -*- coding: utf-8 -*-
#pip install requests beautifulsoup4 lxml selenium

In [8]:
import warnings
warnings.filterwarnings("ignore")

### 저장 경로 및 크롤링 사이트 지정 / html 구조 분석 

In [32]:
import requests
from bs4 import BeautifulSoup
import re
from selenium import webdriver
import os
import time

#저장 경로 
save_path = "./data/smart_factory_crawling_result.csv"
# 크롤링할 사이트 주소를 정의합니다.
source_url = "https://www.dbpia.co.kr/search/topSearch?startCount=0&collection=ALL&range=A&searchField=ALL&sort=RANK&query=%EC%8A%A4%EB%A7%88%ED%8A%B8+%ED%8C%A9%ED%86%A0%EB%A6%AC&srchOption=*&includeAr=false#none"

#내용이 자바스크립트로 실행되기 때문에, 스크립트가 실행된 후 크롤링 시도
driver_path = os.path.join("./chromedriver_win32/", "chromedriver")
driver = webdriver.Chrome(driver_path, options=webdriver.ChromeOptions().add_argument("headless"))
driver.get(source_url)

# 더보기 클릭하여 모든 목록 불러오기
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')
searchCount = soup.find("span", "searchCount").text #검색 결과 숫자로 반복할 횟수 정함
count = re.sub(r'[^0-9]', '', searchCount)
count = int(count) // 10 + 1 

SCROLL_PAUSE_TIME = 1.5 
for index in range(count):
    try :
        driver.find_element_by_xpath("//div[@class='listFoot']/div[@class='paginate']/a[@class='viewMore']").click() 
        time.sleep(SCROLL_PAUSE_TIME) 
    except :  
        break 
    
# 사이트의 html 구조에 기반하여 크롤링을 수행합니다.
html = driver.page_source
driver.close()
soup = BeautifulSoup(html, 'lxml')

div_contRight = soup.find("div", { "class" : "contRight" })

div_listBody = div_contRight.find("div", { "class" : "listBody" })
li_item = div_listBody.find_all("li", { "class" : "item" })
print("item size : " + str(len(li_item)))

item size : 564


### 데이처 추출하고 저장

In [33]:
import pandas as pd #데이터 프레임으로 저장 
import random
import numpy as np

# 크롤링한 데이터를 데이터 프레임으로 만들기 위해 준비합니다.
columns = ['Type', 'Title', 'Author', 'Year', 'Abstract', 'Link']
df = pd.DataFrame(columns=columns)

#초록 추출
def get_abstract(url):
    abstractText = ""    
    if link : 
        driver_path = os.path.join("./chromedriver_win32/", "chromedriver")
        driver = webdriver.Chrome(driver_path, options=webdriver.ChromeOptions().add_argument("headless"))
        driver.get(link)

        html = driver.page_source
        soup2 = BeautifulSoup(html, 'lxml')
        
        abstractDiv = soup2.find("div", { "class" : "abstractWrap" })
        articleP = abstractDiv.find_all("p", { "class" : "article" })
        
        if articleP :
            for article in articleP :
                while True :
                    print( "--------" ,  article.span)
                    if article.span :
                        article.span.decompose() 
                    else :
                        break;
                abstractText = abstractText + article.text
        driver.close()
    
    return abstractText

page_url_base = "https://www.dbpia.co.kr"
for index in range(len(li_item)):
    try : #타입 
        data_type = li_item[index].find("ul", { "class" : "type" }).find("li", { "class" : "data" }).text
    except: 
        data_type = ""
        
    try : #제목
        title = li_item[index].find("div", { "class" : "titWrap" }).find("h5").text
    except: 
        title = ""
       
    try : #저자
        author = li_item[index].find("li", {"class" : "author"}).text
    except: 
        author = ""
       
    try : #발행연도
        year = li_item[index].find("li", {"class" : "date"}).text
    except: 
        year = ""
        
    try : #링크, 초록
        titUrl = li_item[index].find("a").get('href')
        link = page_url_base + titUrl
        abstract = get_abstract(link)
    except Exception as e :
        print('오류 발생~!.', e)
        link = ""
        abstract = ""  
       
    row = [data_type, title, author, year, abstract, link]
    series = pd.Series(row, index=df.columns)
    df = df.append(series, ignore_index=True)
    
    #속도 조절
    rand_value = np.random.randint(1, 5)
    time.sleep(rand_value)
       
df.to_csv(save_path, encoding='utf-8')


-------- None
-------- <span class="newKeyword">
<!-- <dd class="word"> -->
<a href="#none" onclick="javascript:fnSearchKeyword('MQTT(message queuing telemetry transport)','_blank');fn_statistics('Z307', '', '')">#MQTT(message queuing telemetry transport)</a>
<!-- </dd> -->
<!-- <dd class="word"> -->
<a href="#none" onclick="javascript:fnSearchKeyword('AI-smart sensor','_blank');fn_statistics('Z307', '', '')">#AI-smart sensor</a>
<!-- </dd> -->
<!-- <dd class="word"> -->
<a href="#none" onclick="javascript:fnSearchKeyword('SMIC(smart industrial-complex manufacturing innovation center)','_blank');fn_statistics('Z307', '', '')">#SMIC(smart industrial-complex manufacturing innovation center)</a>
<!-- </dd> -->
<!-- <dd class="word"> -->
<a href="#none" onclick="javascript:fnSearchKeyword('RTD(resistive temperature device)','_blank');fn_statistics('Z307', '', '')">#RTD(resistive temperature device)</a>
<!-- </dd> -->
<!-- <dd class="word"> -->
<a href="#none" onclick="javascript:fnSearchKe

-------- <span class="nodata">
<span>등록된 정보가 없습니다.</span>
</span>
-------- None
-------- <span class="nodata">
<span>등록된 정보가 없습니다.</span>
</span>
-------- <span class="newKeyword">
<!-- <dd class="word"> -->
<a href="#none" onclick="javascript:fnSearchKeyword('Standard Uncertainty','_blank');fn_statistics('Z307', '', '')">#Standard Uncertainty</a>
<!-- </dd> -->
<!-- <dd class="word"> -->
<a href="#none" onclick="javascript:fnSearchKeyword('Dynamic Properties','_blank');fn_statistics('Z307', '', '')">#Dynamic Properties</a>
<!-- </dd> -->
<!-- <dd class="word"> -->
<a href="#none" onclick="javascript:fnSearchKeyword('GIGA Steel','_blank');fn_statistics('Z307', '', '')">#GIGA Steel</a>
<!-- </dd> -->
<!-- <dd class="word"> -->
<a href="#none" onclick="javascript:fnSearchKeyword('Smart Factory','_blank');fn_statistics('Z307', '', '')">#Smart Factory</a>
<!-- </dd> -->
<!-- <dd class="word"> -->
<a href="#none" onclick="javascript:fnSearchKeyword('Automation','_blank');fn_statistics('Z307

-------- <span class="nodata">
<span>등록된 정보가 없습니다.</span>
</span>
-------- None
-------- <span class="nodata">
<span>등록된 정보가 없습니다.</span>
</span>
-------- None
-------- None
-------- <span class="newKeyword">
<!-- <dd class="word"> -->
<a href="#none" onclick="javascript:fnSearchKeyword('스마트팩토리','_blank');fn_statistics('Z307', '', '')">#스마트팩토리</a>
<!-- </dd> -->
<!-- <dd class="word"> -->
<a href="#none" onclick="javascript:fnSearchKeyword('통신 프로토콜','_blank');fn_statistics('Z307', '', '')">#통신 프로토콜</a>
<!-- </dd> -->
<!-- <dd class="word"> -->
<a href="#none" onclick="javascript:fnSearchKeyword('Smart Factroy','_blank');fn_statistics('Z307', '', '')">#Smart Factroy</a>
<!-- </dd> -->
<!-- <dd class="word"> -->
<a href="#none" onclick="javascript:fnSearchKeyword('Modbus','_blank');fn_statistics('Z307', '', '')">#Modbus</a>
<!-- </dd> -->
<!-- <dd class="word"> -->
<a href="#none" onclick="javascript:fnSearchKeyword('PLC','_blank');fn_statistics('Z307', '', '')">#PLC</a>
<!-- </dd> -->
<

-------- <span class="nodata">
<span>등록된 정보가 없습니다.</span>
</span>
-------- None
-------- <span class="nodata">
<span>등록된 정보가 없습니다.</span>
</span>
-------- None
-------- <span class="nodata">
<span>등록된 정보가 없습니다.</span>
</span>
-------- None
-------- <span class="nodata">
<span>등록된 정보가 없습니다.</span>
</span>
-------- <span class="newKeyword">
<!-- <dd class="word"> -->
<a href="#none" onclick="javascript:fnSearchKeyword('manufacturing data','_blank');fn_statistics('Z307', '', '')">#manufacturing data</a>
<!-- </dd> -->
<!-- <dd class="word"> -->
<a href="#none" onclick="javascript:fnSearchKeyword('assistant','_blank');fn_statistics('Z307', '', '')">#assistant</a>
<!-- </dd> -->
<!-- <dd class="word"> -->
<a href="#none" onclick="javascript:fnSearchKeyword('real time monitoring','_blank');fn_statistics('Z307', '', '')">#real time monitoring</a>
<!-- </dd> -->
<!-- <dd class="word"> -->
<a href="#none" onclick="javascript:fnSearchKeyword('smart factory','_blank');fn_statistics('Z307', '', ''

-------- <span class="nodata">
<span>등록된 정보가 없습니다.</span>
</span>
-------- <span class="newKeyword">
<!-- <dd class="word"> -->
<a href="#none" onclick="javascript:fnSearchKeyword('Pallet','_blank');fn_statistics('Z307', '', '')">#Pallet</a>
<!-- </dd> -->
<!-- <dd class="word"> -->
<a href="#none" onclick="javascript:fnSearchKeyword('Lift','_blank');fn_statistics('Z307', '', '')">#Lift</a>
<!-- </dd> -->
<!-- <dd class="word"> -->
<a href="#none" onclick="javascript:fnSearchKeyword('Smart Factory','_blank');fn_statistics('Z307', '', '')">#Smart Factory</a>
<!-- </dd> -->
<!-- <dd class="word"> -->
<a href="#none" onclick="javascript:fnSearchKeyword('Position','_blank');fn_statistics('Z307', '', '')">#Position</a>
<!-- </dd> -->
</span>
-------- None
-------- <span class="nodata">
<span>등록된 정보가 없습니다.</span>
</span>
-------- None
-------- <span class="nodata">
<span>등록된 정보가 없습니다.</span>
</span>
-------- <span class="newKeyword">
<!-- <dd class="word"> -->
<a href="#none" onclick="javasc

...

오류 발생~!. 'NoneType' object has no attribute 'find_all'
오류 발생~!. 'NoneType' object has no attribute 'find_all'
-------- None
-------- <span class="newKeyword">
</span>
-------- None
-------- <span class="nodata">
<span>등록된 정보가 없습니다.</span>
</span>
-------- None
-------- None
-------- <span class="newKeyword">
</span>
-------- None
-------- None
-------- <span class="newKeyword">
</span>
-------- None
오류 발생~!. 'NoneType' object has no attribute 'find_all'
-------- None
-------- <span class="newKeyword">
</span>
-------- None
-------- None
-------- <span class="newKeyword">
</span>
-------- None
-------- None
-------- <span class="newKeyword">
</span>
-------- None
-------- <span class="nodata">
<span>등록된 정보가 없습니다.</span>
</span>
-------- None
-------- <span class="nodata">
<span>등록된 정보가 없습니다.</span>
</span>
-------- None
-------- None
-------- <span class="newKeyword">
<!-- <dd class="word"> -->
<a href="#none" onclick="javascript:fnSearchKeyword('산업 무선 센서 네트워크','_blank');fn_statistics('

-------- None
-------- <span class="newKeyword">
<!-- <dd class="word"> -->
<a href="#none" onclick="javascript:fnSearchKeyword('MQTT','_blank');fn_statistics('Z307', '', '')">#MQTT</a>
<!-- </dd> -->
<!-- <dd class="word"> -->
<a href="#none" onclick="javascript:fnSearchKeyword('Node-RED','_blank');fn_statistics('Z307', '', '')">#Node-RED</a>
<!-- </dd> -->
<!-- <dd class="word"> -->
<a href="#none" onclick="javascript:fnSearchKeyword('NodeMCU','_blank');fn_statistics('Z307', '', '')">#NodeMCU</a>
<!-- </dd> -->
<!-- <dd class="word"> -->
<a href="#none" onclick="javascript:fnSearchKeyword('Monitoring system','_blank');fn_statistics('Z307', '', '')">#Monitoring system</a>
<!-- </dd> -->
</span>
-------- None
-------- None
-------- <span class="newKeyword">
<!-- <dd class="word"> -->
<a href="#none" onclick="javascript:fnSearchKeyword('VLC','_blank');fn_statistics('Z307', '', '')">#VLC</a>
<!-- </dd> -->
<!-- <dd class="word"> -->
<a href="#none" onclick="javascript:fnSearchKeyword('AG

-------- <span class="nodata">
<span>등록된 정보가 없습니다.</span>
</span>
-------- <span class="newKeyword">
<!-- <dd class="word"> -->
<a href="#none" onclick="javascript:fnSearchKeyword('제4차 산업혁명','_blank');fn_statistics('Z307', '', '')">#제4차 산업혁명</a>
<!-- </dd> -->
<!-- <dd class="word"> -->
<a href="#none" onclick="javascript:fnSearchKeyword('산업 4.0','_blank');fn_statistics('Z307', '', '')">#산업 4.0</a>
<!-- </dd> -->
<!-- <dd class="word"> -->
<a href="#none" onclick="javascript:fnSearchKeyword('스마트팩토리','_blank');fn_statistics('Z307', '', '')">#스마트팩토리</a>
<!-- </dd> -->
<!-- <dd class="word"> -->
<a href="#none" onclick="javascript:fnSearchKeyword('인공지능','_blank');fn_statistics('Z307', '', '')">#인공지능</a>
<!-- </dd> -->
<!-- <dd class="word"> -->
<a href="#none" onclick="javascript:fnSearchKeyword('사물인터넷','_blank');fn_statistics('Z307', '', '')">#사물인터넷</a>
<!-- </dd> -->
</span>
-------- None
-------- None
-------- <span class="newKeyword">
</span>
-------- None
-------- None
-------- <span

-------- None
-------- <span class="newKeyword">
</span>
-------- None
-------- None
-------- <span class="newKeyword">
<!-- <dd class="word"> -->
<a href="#none" onclick="javascript:fnSearchKeyword('IoT','_blank');fn_statistics('Z307', '', '')">#IoT</a>
<!-- </dd> -->
<!-- <dd class="word"> -->
<a href="#none" onclick="javascript:fnSearchKeyword('정보통신공사업','_blank');fn_statistics('Z307', '', '')">#정보통신공사업</a>
<!-- </dd> -->
<!-- <dd class="word"> -->
<a href="#none" onclick="javascript:fnSearchKeyword('3차원 미래예측','_blank');fn_statistics('Z307', '', '')">#3차원 미래예측</a>
<!-- </dd> -->
<!-- <dd class="word"> -->
<a href="#none" onclick="javascript:fnSearchKeyword('STEPPER','_blank');fn_statistics('Z307', '', '')">#STEPPER</a>
<!-- </dd> -->
<!-- <dd class="word"> -->
<a href="#none" onclick="javascript:fnSearchKeyword('스마트 팩토리','_blank');fn_statistics('Z307', '', '')">#스마트 팩토리</a>
<!-- </dd> -->
<!-- <dd class="word"> -->
<a href="#none" onclick="javascript:fnSearchKeyword('Information and 

-------- None
-------- <span class="newKeyword">
</span>
-------- None
-------- None
-------- <span class="newKeyword">
</span>
-------- None
-------- None
-------- <span class="newKeyword">
</span>
-------- None
-------- <span class="nodata">
<span>등록된 정보가 없습니다.</span>
</span>
-------- None
